Synopsis:  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymongo, string, re
import gridfs

#### Obtaining the Data Files

The Consolidated Federal Funds data set is available for download at :  
http://www2.census.gov/pub/outgoing/govs/special60/

In [27]:
import requests

In [28]:
files = requests.get('http://www2.census.gov/pub/outgoing/govs/special60/')

In [30]:
files.status_code

200

In [34]:
import re

In [40]:
zips = re.compile('href=".*\.zip"')

In [41]:
zips.findall(files.text)

['href="1-3_Govt_Org_Nat_CoArea_ElecOff.zip"',
 'href="1_Emp_Retire_Historical_DB.zip"',
 'href="4_Govt_Org_Directory_Surveys.zip"',
 'href="City_Govt_Emp.zip"',
 'href="City_Govt_Fin.zip"',
 'href="County_Area_Emp.zip"',
 'href="County_Area_Fin.zip"',
 'href="County_Govt_Emp.zip"',
 'href="County_Govt_Fin.zip"',
 'href="Govt_Finances.zip"',
 'href="Public_Emp.zip"',
 'href="Small_Municipal_and_Twp_Gov_Finances.zip"',
 'href="State_Govt_Fin.zip"',
 'href="_IndEmp1972-2010.zip"',
 'href="_IndFin_1967-2012.zip"',
 'href="_Tax_Prop_Value_DB.zip"',
 'href="consolidated%20federal%20funds%20report%201983-1992.zip"',
 'href="consolidated%20federal%20funds%20report%201993-2010.zip"',
 'href="consolidated%20federal%20funds%20report%20states%201993-2010.zip"',
 'href="county_area_finances_2007.zip"',
 'href="county_area_finances_2012.zip"',
 'href="elsec%20school%20district%20finance%20data%20fy%201987-91.zip"',
 'href="emp_est.zip"',
 'href="hist_emp.zip"',
 'href="hist_fin.zip"',
 'href="indfi

Everything we need for this report is in the Consolidated Federal Funds Report zip

In [46]:
with open('./data/consolidated federal funds report states 1993-2010.zip', 'wb') as f:
    response = requests.get('http://www2.census.gov/pub/outgoing/govs/special60/consolidated%20federal%20funds%20report%20states%201993-2010.zip')
    file_out = response.raw
    f.write(file_out.read())

# Opening the  raw  files

In [2]:
with open('./data/consolidated federal funds report 1993-2010/10agen.txt', 'r') as f: 
    agencies_raw = f.readlines()
with open('./data/consolidated federal funds report 1993-2010/10prog.txt', 'r') as f: 
    programs_raw = f.readlines()
with open('./data/consolidated federal funds report 1993-2010/10cffcom.txt', 'r') as f: 
    data_raw = f.readlines()

#### Efficiently parsing the data files

In [3]:
def sliceAgencyCodes(s):
    result = {}
    result['AGENCY_CODE'] = s[0:4]
    result['AGENCY_NAME'] = s[4:].strip()
    return result
def sliceFundingReport(s):
    result = {}
    result['FIPS_CODE'] = s[:10]
    result['STATE_CODE'] = s[:2]
    result['COUNTY_CODE'] = s[3:6]
    result['PLACE_CODE'] = s[5:10]
    result['STATE'] = s[10:12]
    result['COUNTY'] = s[12:36].strip()
    result['PLACE'] = s[36:60].strip()
    result['POP'] = s[60:69]
    result['CONG_DIST'] = s[69:103].strip()
    result['PROG'] = s[103:109]
    result['OBJ'] = s[109:111]
    result['AGENCY'] = s[111:115]
    result['SIGN'] = s[115]
    result['AMOUNT'] = s[116:129].strip()
    return result
def sliceProgramsReport(s):
    result = {}
    result['CODE'] = s[:6]
    result['NAME'] = s[6:].strip()
    return result

In [4]:
sliceProgramsReport(programs_raw[0])

{'CODE': '10.001', 'NAME': 'AGRICULTURAL RESEARCH-BASIC AND APPLIED RESEARCH'}

In [5]:
sliceAgencyCodes(agencies_raw[0])

{'AGENCY_CODE': '0000', 'AGENCY_NAME': 'LEGISLATIVE BRANCH'}

In [6]:
sliceFundingReport(data_raw[0])

{'AGENCY': '7022',
 'AMOUNT': '000002220055',
 'CONG_DIST': '02',
 'COUNTY': 'Autauga County',
 'COUNTY_CODE': '010',
 'FIPS_CODE': '0100103220',
 'OBJ': 'II',
 'PLACE': 'Autaugaville',
 'PLACE_CODE': '03220',
 'POP': '000000000',
 'PROG': '97.022',
 'SIGN': '0',
 'STATE': 'AL',
 'STATE_CODE': '01'}

#### Creating the DataFrames and Dictionaries

In [7]:
def createProgramsDict(obj):
    programs = {}
    for line in obj:
        result = sliceProgramsReport(line)
        programs[result['CODE']] = result['NAME']
    return programs
def createAgenciesDict(obj):
    agencies = {}
    for line in obj:
        result = sliceAgencyCodes(line)
        agencies[result['AGENCY_CODE']] = result['AGENCY_NAME']
    return agencies
def createFundingReportDict(obj):
    data = {}
    counter = 0
    for line in obj:
        result = sliceFundingReport(line)
        data[counter] = result
        counter += 1
    return data

In [8]:
Programs = createProgramsDict(programs_raw)

In [9]:
Programs['97.022']

'FLOOD INSURANCE'

In [10]:
Agencies = createAgenciesDict(agencies_raw)

In [11]:
Agencies['7022']

'FEDERAL EMERGENCY MANAGEMENT AGENCY'

In [12]:
data = createFundingReportDict(data_raw)

In [13]:
len(data_raw), len(data)

(539075, 539075)

In [14]:
data[1]

{'AGENCY': '1700',
 'AMOUNT': '000000026000',
 'CONG_DIST': '02',
 'COUNTY': 'Autauga County',
 'COUNTY_CODE': '010',
 'FIPS_CODE': '0100103220',
 'OBJ': 'DR',
 'PLACE': 'Autaugaville',
 'PLACE_CODE': '03220',
 'POP': '000000000',
 'PROG': 'DR.100',
 'SIGN': '0',
 'STATE': 'AL',
 'STATE_CODE': '01'}

In [15]:
df_data = pd.DataFrame.from_dict(data, orient='index')

In [16]:
df_data.head(3)

,CONG_DIST,PLACE_CODE,STATE,AGENCY,COUNTY,PROG,SIGN,PLACE,AMOUNT,COUNTY_CODE,FIPS_CODE,OBJ,POP,STATE_CODE
0,02,03220,AL,7022,Autauga County,97.022,0,Autaugaville,000002220055,010,0100103220,II,000000000,01
1,02,03220,AL,1700,Autauga County,DR.100,0,Autaugaville,000000026000,010,0100103220,DR,000000000,01
2,02,03220,AL,1727,Autauga County,DR.100,0,Autaugaville,000000005000,010,0100103220,DR,000000000,01


In [25]:
import copy

In [26]:
df_data2 = copy.deepcopy(df_data)

In [18]:
df_data2['AGENCY_NAME'] = 'NA'
df_data2['PROGRAM_NAME'] = 'NA'

In [19]:
df_data2.head(1)

,CONG_DIST,PLACE_CODE,STATE,AGENCY,COUNTY,PROG,SIGN,PLACE,AMOUNT,COUNTY_CODE,FIPS_CODE,OBJ,POP,STATE_CODE,AGENCY_NAME,PROGRAM_NAME
0,02,03220,AL,7022,Autauga County,97.022,0,Autaugaville,000002220055,010,0100103220,II,000000000,01,NA,NA


In [20]:
def lookupAgency(s):
    return Agencies[s]
def lookupProgram(s):
    return Programs[s]

#### Joining the Data

In [21]:
df_data2['PROGRAM_NAME'] = df_data2.apply(lambda x: lookupProgram(x['PROG']), axis=1)

In [22]:
df_data2['AGENCY_NAME'] = df_data2.apply(lambda x: lookupAgency(x['AGENCY']), axis=1)

In [23]:
df_data2.head(1)

,CONG_DIST,PLACE_CODE,STATE,AGENCY,COUNTY,PROG,SIGN,PLACE,AMOUNT,COUNTY_CODE,FIPS_CODE,OBJ,POP,STATE_CODE,AGENCY_NAME,PROGRAM_NAME
0,02,03220,AL,7022,Autauga County,97.022,0,Autaugaville,000002220055,010,0100103220,II,000000000,01,FEDERAL EMERGENCY MANAGEMENT AGENCY,FLOOD INSURANCE


In [24]:
df_data = df_data2

#### Writing to Mongo

In [198]:
client = pymongo.MongoClient()

In [200]:
db = client.OpenData_Government_Funding

In [233]:
year2010Combined = db.create_collection('year2010Combined')

In [203]:
import json

In [229]:
df_json = json.loads(df_data.T.to_json())

In [230]:
df_json['17487']

{'AGENCY': '1630',
 'AGENCY_NAME': 'EMPLOYMENT AND TRAINING ADMINISTRATION',
 'AMOUNT': '000012822660',
 'CONG_DIST': '0203040507',
 'COUNTY': 'Maricopa County',
 'COUNTY_CODE': '135',
 'FIPS_CODE': '0401355000',
 'OBJ': 'GG',
 'PLACE': 'Phoenix',
 'PLACE_CODE': '55000',
 'POP': '000000000',
 'PROG': '17.207',
 'PROGRAM_NAME': 'EMPLOYMENT SERVICE',
 'SIGN': '0',
 'STATE': 'AZ',
 'STATE_CODE': '04'}

In [234]:
for record in df_json:
    db.year2010Combined.insert_one(df_json[record])

#### Running a Report

In [250]:
B = db.year2010Combined.find({'AGENCY':'9999'})

In [251]:
pd.DataFrame(list(B))

,AGENCY,AGENCY_NAME,AMOUNT,CONG_DIST,COUNTY,COUNTY_CODE,FIPS_CODE,OBJ,PLACE,PLACE_CODE,POP,PROG,PROGRAM_NAME,SIGN,STATE,STATE_CODE,_id
0,9999,UNNAMED AGENCY-NONDEFENSE,000151658888,,U.S. undistributed,999,9999999999,PC,U.S. undistributed,99999,000000000,PC.200,PROCUREMENT CONTRACTS--ALL FED GOVT AGENCIES O...,0,UD,99,57dcddbb73cf75271e013da0


In [268]:
df_data.AMOUNT.astype(int).quantile([.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,.99]).astype(int)

0.10         5698
0.20        18000
0.30        42000
0.40        87244
0.50       173000
0.60       345256
0.70       738253
0.80      1900000
0.90      7236216
0.95     21341194
0.99    138667542
Name: AMOUNT, dtype: int64

In [273]:
df_data['AMOUNT'] = np.array(df_data['AMOUNT']).astype(int)

In [274]:
agency_spending = df_data.groupby('AGENCY_NAME').agg({'AMOUNT':np.sum})

In [283]:
agency_spending.AMOUNT.sort_values(ascending=False).head(5)

AGENCY_NAME
FEDERAL EMERGENCY MANAGEMENT AGENCY                                  1242911715358
CENTERS FOR MEDICARE AND MEDICAID SERVICES                            812386547077
SOCIAL SECURITY ADMINISTRATION                                        749199692970
ASSISTANT SECRETARY FOR HOUSING--FEDERAL HOUSING COMMISSIONER         306543577748
ARMY, DEPARTMENT OF THE (EXCEPT CORPS OF ENGINEERS CIVIL PROGRAM)     263709200970
Name: AMOUNT, dtype: int64

In [284]:
agency_spending.AMOUNT.sort_values(ascending=True).head(5)

AGENCY_NAME
U.S ARMY CORPS OF ENGINEERS - CIVIL PROGRAM FINANCING ONLY     20000
RECOVERY ACCOUNTABILITY AND TRANSPARENCY BOARD                 72361
WOODROW WILSON INTERNATIONAL CENTER FOR SCHOLARS              106824
NATIONAL SECURITY AGENCY/CENTRAL SECURITY SERVICE             150000
OFFICE OF FEDERAL HOUSING ENTERPRISE OVERSIGHT                215638
Name: AMOUNT, dtype: int64

In [285]:
program_spending = df_data.groupby('PROGRAM_NAME').agg({'AMOUNT':np.sum})

In [289]:
program_spending.AMOUNT.sort_values(ascending=True).count()

1588

In [292]:
test = df_data['PROGRAM_NAME']  

In [291]:
program_spending.head(10)

,AMOUNT
PROGRAM_NAME,
1890 INSTITUTION CAPACITY BUILDING GRANTS,32514073
1994 INSTITUTIONS RESEARCH PROGRAM,1772911
21ST CENTURY COMMUNITY LEARNING CENTERS,1142842680
21ST CENTURY MUSEUM PROFESSIONALS,1795798
ABANDONED INFANTS,10900315
ABANDONED MINE HAZARD MITIGATION - RECOVERY,406530
ABANDONED MINE LAND RECLAMATION PROGRAM,598924114
ABSTINENCE EDUCATION,22905173
ACADEMIC EXCHANGE PROGRAMS - HUMPHREY FELLOWSHIP PROGRAM,13538425


In [293]:
test

'21ST CENTURY COMMUNITY LEARNING CENTERS'

More to come